In [18]:
import rasterio 
import geopandas as gpd
import numpy as np
from osgeo import gdal
import matplotlib.pyplot as plt
import rasterio as rs
from rasterio.plot import show
import georasters as gr



In [15]:
dat = rasterio.open("DHMVIIDSMRAS1m_k13/GeoTIFF")
profile = dat.profile.copy()
profile.update(compress='lzw')

with rasterio.open("testsample_compressed.tif", 'w', **profile) as dst:
    for ji, window in dat.block_windows(1):
     dst.write(dat.read(window=window), window=window)
    
print(window)
print(profile)

Window(col_off=0, row_off=6784, width=5022, height=1)
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 5022, 'height': 6785, 'count': 1, 'crs': CRS.from_epsg(32617), 'transform': Affine(2.4384, 0.0, 730232.51043,
       0.0, -2.4384, 4557035.326668), 'tiled': False, 'compress': 'lzw', 'interleave': 'band'}


In [19]:
# Load data
raster = 'DHMVIIDSMRAS1m_k13.tif'
data = gr.from_file(raster)


In [16]:
from rasterio.transform import from_origin
with rasterio.open(
        'Resample.tif', 'w', driver='GTiff', dtype='uint16',
        width=720, height=360, count=8, crs='EPSG:4326',
        transform=from_origin(-180.0, 90.0, 0.5, 0.5),
        nodata=0, tiled=True, compress='lzw') as dataset:
    dataset.write()
    

TypeError: write() takes at least 1 positional argument (0 given)

In [44]:
import os
import numpy as np
import dask.array as da

In [55]:
INPUTPATH = os.path.join('Dsm13/InputpathGeotiff')

OUTPUTPATH = os.path.join('Dsm13/outputpathGeotiff')



In [2]:
def readFile(filename):
    filehandle = gdal.Open(filename)
    band1 = filehandle.GetRasterBand(1)
    geotransform = filehandle.GetGeoTransform()
    geoproj = filehandle.GetProjection()
    Z = band1.ReadAsArray() # Can this be avoided?
    xsize = filehandle.RasterXSize
    ysize = filehandle.RasterYSize
    filehandle = None
    return xsize,ysize,geotransform,geoproj,Z

def writeFile(filename,geotransform,geoprojection,data):
    (x,y) = data.shape
    format = "GTiff"
    driver = gdal.GetDriverByName(format)
    # you can change the dataformat but be sure to be able to store negative values including -9999
    dst_datatype = gdal.GDT_Float32
    dst_ds = driver.Create(filename,y,x,1,dst_datatype, [ 'COMPRESS=LZW' ])
    dst_ds.GetRasterBand(1).SetNoDataValue(-9999)
    dst_ds.GetRasterBand(1).WriteArray(data)
    dst_ds.SetGeoTransform(geotransform)
    dst_ds.SetProjection(geoprojection)
    dst_ds = None
    return 1

files = os.listdir(INPUTPATH)

for oneFile in files:
    print(oneFile)
    InputBaseName = oneFile.split('.')[0]
    [xsize, ysize, geotransform, geoproj, Z] = readFile(os.path.join(INPUTPATH, oneFile))
    Z[Z<-9000] = -9999
    writefilename = os.path.join(OUTPUTPATH,(InputBaseName+".tif"))
    writeFile(writefilename, geotransform, geoproj, Z)

# print("Done")
img = r'DHMVIIDSMRAS1m_k13.tif'.readFile
show(img)

NameError: name 'os' is not defined